# **Mulit-DOC Reading Chatbot using RAG and** **Streamlit**

## Install required libraries

In [1]:

!pip install -q streamlit pymupdf chromadb pyngrok faiss-cpu sentence-transformers mistralai==0.4.2 python-docx streamlit-copy-to-clipboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 87.4 MB/s eta

In [2]:
!pip install -U langchain
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
pip show streamlit


Name: streamlit
Version: 1.49.1
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: streamlit-copy-to-clipboard


## Required API keys and ngrok token

In [4]:
from google.colab import userdata

NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
MISTRAL_API_KEY  = userdata.get('MISTRAL_API_KEY')

if NGROK_AUTH_TOKEN is None:
    print("❗ NGROK_AUTH_TOKEN missing in Colab secrets.")
if MISTRAL_API_KEY is None:
    print("❗ MISTRAL_API_KEY missing in Colab secrets.")

if NGROK_AUTH_TOKEN:
    !ngrok config add-authtoken $NGROK_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## User-interface web-app

In [20]:
%%writefile app.py
import os, tempfile
import streamlit as st
import fitz
from docx import Document
import numpy as np
import pandas as pd
from collections import Counter
from typing import List

from sentence_transformers import SentenceTransformer, util
import chromadb
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

# ---------- Constants ----------
FALLBACK_ANSWER = "I could not find the answer in the uploaded documents."
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "doc_chunks"

# ---------- Page Config ----------
st.set_page_config(page_title="Multi-Document Chatbot with Dashboard", layout="wide")
st.title("🤖 Multi-Document Chatbot with Dashboard")

# ---------- Session State Initialization ----------
def ensure_state():
    ss = st.session_state
    ss.setdefault("db", None)
    ss.setdefault("chunks", [])
    ss.setdefault("doc_names", [])
    ss.setdefault("history", [])
    ss.setdefault("answered_questions", set())
    ss.setdefault("embedder", SentenceTransformer(EMBED_MODEL))
    ss.setdefault("collection_name", COLLECTION_NAME)
    ss.setdefault("top_k", 4)
    ss.setdefault("chunk_size", 2000)
    ss.setdefault("chunk_overlap", 400)
    ss.setdefault("grounding_threshold", 0.55)
    ss.setdefault("sim_threshold", 0.55)
    return ss

ss = ensure_state()

# ---------- Sidebar ----------
with st.sidebar:
    st.header("1) Upload documents")
    files = st.file_uploader(
        "PDF / DOCX / TXT", type=["pdf", "docx", "txt"],
        accept_multiple_files=True, label_visibility="collapsed"
    )

    st.header("2) Processing Parameters")
    ss.top_k = st.slider("Top-k retrieved chunks", 1, 12, ss.top_k, 1)
    ss.chunk_size = st.number_input("Chunk size (chars)", 500, 8000, ss.chunk_size, 100)
    ss.chunk_overlap = st.number_input("Chunk overlap (chars)", 0, 4000, ss.chunk_overlap, 50)

    st.markdown("---")
    st.header("3) RAG/Performance Metrics")
    ss.grounding_threshold = st.slider("Groundedness threshold", 0.1, 0.95, ss.grounding_threshold)
    ss.sim_threshold = st.slider("Relevant-chunk sim threshold", 0.1, 0.95, ss.sim_threshold)

    st.markdown("---")
    process_btn = st.button("Process Documents", use_container_width=True)
    clear_btn = st.button("Clear Chat History", use_container_width=True)

    st.markdown("---")
    st.caption("API keys via environment variables (set MISTRAL_API_KEY)")
    st.code("os.environ['MISTRAL_API_KEY']", language="python")

if clear_btn:
    ss.history = []
    ss.answered_questions = set()
    st.experimental_rerun()

# ---------- Utility Functions ----------
def extract_text(uploaded_file, file_type: str) -> str:
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=f".{file_type}") as tmp:
            tmp.write(uploaded_file.getbuffer())
            path = tmp.name
        if file_type == "pdf":
            with fitz.open(path) as doc:
                text = "".join(page.get_text() for page in doc)
        elif file_type == "docx":
            d = Document(path)
            text = "\n".join([p.text for p in d.paragraphs if p.text.strip()])
        elif file_type == "txt":
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()
        else:
            text = ""
        os.remove(path)
        return text
    except Exception as e:
        st.error(f"Error reading {uploaded_file.name}: {e}")
        return ""

def chunk_text(text: str, size: int, overlap: int) -> List[str]:
    step = max(size - overlap, 1)
    return [text[i:i + size] for i in range(0, len(text), step)]

def cosine_sim(a, b) -> float:
    return float(util.cos_sim(a, b).cpu().numpy().squeeze())

def overlap_coverage(answer: str, context: str) -> float:
    ans_tokens = answer.lower().split()
    ctx_tokens = context.lower().split()
    if not ctx_tokens:
        return 0.0
    overlap = sum((Counter(ans_tokens) & Counter(ctx_tokens)).values())
    return overlap / len(ans_tokens) if ans_tokens else 0.0

def precision_recall_proxies(query_vec, doc_vecs: np.ndarray, sims: List[float], sim_thr: float):
    sims_arr = np.array(sims) if len(sims) else np.array([])
    if sims_arr.size == 0:
        return 0.0, 0.0
    precision = float((sims_arr >= sim_thr).sum()) / len(sims_arr)
    recall = float(sims_arr.max())
    return precision, recall

def timed(fn, *a, **kw):
    import time
    t0 = time.time()
    out = fn(*a, **kw)
    return out, time.time() - t0

# ---------- Document Processing ----------
if files and process_btn:
    with st.spinner("Processing documents..."):
        ss.chunks, ss.doc_names = [], []
        for f in files:
            ext = f.name.split(".")[-1].lower()
            text = extract_text(f, ext)
            if not text:
                continue
            pieces = chunk_text(text, int(ss.chunk_size), int(ss.chunk_overlap))
            ss.chunks.extend(pieces)
            ss.doc_names.extend([f.name]*len(pieces))

        if ss.chunks:
            embedder = SentenceTransformer(EMBED_MODEL)
            ss.embedder = embedder
            embeddings = embedder.encode(ss.chunks, show_progress_bar=True)

            client = chromadb.PersistentClient(path="./chroma_db")
            if ss.collection_name in [c.name for c in client.list_collections()]:
                client.delete_collection(ss.collection_name)
            col = client.create_collection(ss.collection_name, metadata={"hnsw:space":"cosine"})
            col.add(
                documents=ss.chunks,
                embeddings=embeddings.tolist(),
                ids=[str(i) for i in range(len(ss.chunks))],
                metadatas=[{"doc": n} for n in ss.doc_names],
            )
            ss.db = col
            ss.history = []
            ss.answered_questions = set()
            st.success(f"Processed {len(files)} document(s).")

# ---------- Main Interface Tabs ----------
tabs = st.tabs(["Chat", "Dashboard"])

# ---------- Chat Tab ----------
with tabs[0]:
    MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

    if ss.db is None:
        st.info("Please upload and process documents to begin.")
    elif not MISTRAL_API_KEY:
        st.error("Mistral API key not found in environment. Set MISTRAL_API_KEY.")
    else:
        # Display chat history (user + assistant messages, single loop)
        for i, msg in enumerate(ss.history):
            with st.chat_message(msg["role"]):
                st.markdown(msg["content"])
                if msg["role"] == "assistant" and msg.get("metrics"):
                    m = msg.get("metrics")
                    cols = st.columns(6)
                    cols[0].metric("⏱️ Latency", f"{m.get('total_latency_s', 0):.2f}s")
                    cols[1].metric("🔎 Avg Q–Doc", f"{m.get('avg_q_doc_sim', 0):.2f}")
                    cols[2].metric("🏅 Max Q–Doc", f"{m.get('max_q_doc_sim', 0):.2f}")
                    cols[3].metric("Groundedness", f"{m.get('answer_context_sim', 0):.2f}")
                    cols[4].metric("Relevance", f"{m.get('answer_relevance_sim', 0):.2f}")
                    cols[5].metric("Coverage", f"{m.get('coverage', 0):.2f}")

                    fb_cols = st.columns([1,1,8])
                    if fb_cols[0].button("👍", key=f"up_{i}", disabled=(msg.get("feedback") is not None)):
                        msg["feedback"] = "positive"
                        # optionally add a flag to trigger rerun but avoid duplicate appends
                    if fb_cols[1].button("👎", key=f"down_{i}", disabled=(msg.get("feedback") is not None)):
                        msg["feedback"] = "negative"

                    if msg.get("hallucination_risk"):
                        st.warning("⚠️ Potential low grounding detected (below threshold).")

                    with st.expander("Retrieved chunks & scores"):
                        for j, (doc, sim) in enumerate(zip(msg.get("retrieved_docs", []), msg.get("retrieved_sims", [])), 1):
                            st.markdown(f"**Top {j} • Sim:** {sim:.3f} • **Source:** `{doc['source']}`")
                            st.code(doc["content"][:1000] + ("..." if len(doc["content"]) > 1000 else ""), language="markdown")

        # Chat input at bottom
        user_input = st.chat_input("Ask a question...", key="chat_input")

        if user_input and user_input not in ss.answered_questions:
            ss.answered_questions.add(user_input)
            ss.history.append({"role": "user", "content": user_input})

            with st.chat_message("assistant"):
                with st.spinner("Thinking..."):
                    # Retrieval
                    q_vec, t_emb = timed(ss.embedder.encode, [user_input])
                    res, t_ret = timed(
                        ss.db.query,
                        query_embeddings=q_vec.tolist(),
                        n_results=int(ss.top_k),
                        include=["documents", "distances", "metadatas", "embeddings"]
                    )
                    docs, dist, meta, embs = (
                        res["documents"][0],
                        res.get("distances", [[]])[0],
                        res.get("metadatas", [[]])[0],
                        np.array(res.get("embeddings", [[]])[0], dtype=np.float32) if res.get("embeddings") else np.array([]),
                    )
                    sims = [1.0 - d for d in dist] if dist else []
                    context = "\n\n".join(docs)

                    client = MistralClient(api_key=MISTRAL_API_KEY)
                    messages = [
                        ChatMessage(role="system", content=(
                            "Answer based ONLY on the context. If not present, say you could not find the answer.\n\n"
                            f"Context:\n{context}"
                        )),
                        ChatMessage(role="user", content=user_input),
                    ]
                    resp, t_gen = timed(client.chat, model="mistral-small-latest", messages=messages)
                    answer = resp.choices[0].message.content if resp.choices else FALLBACK_ANSWER

                    ans_vec = ss.embedder.encode([answer])[0]
                    ctx_vec = ss.embedder.encode([context])[0] if context.strip() else np.zeros_like(ans_vec)

                    if embs is not None and len(embs) > 0:
                        avg_embs_vec = np.mean(np.array(embs), axis=0).astype(np.float32)
                        answer_relevance_sim = cosine_sim(ans_vec, avg_embs_vec)
                    else:
                        answer_relevance_sim = 0.0

                    avg_q_doc = float(np.mean(sims)) if sims else 0.0
                    max_q_doc = float(np.max(sims)) if sims else 0.0
                    groundedness = cosine_sim(ans_vec, ctx_vec) if context.strip() else 0.0
                    hallucination_score = 1.0 - groundedness
                    coverage = overlap_coverage(answer, context)
                    ans_len = len(answer.split())

                    context_precision_proxy, context_recall_proxy = precision_recall_proxies(q_vec[0], embs, sims, float(ss.sim_threshold))

                    metrics = {
                        "total_latency_s": float(t_emb + t_ret + t_gen),
                        "avg_q_doc_sim": avg_q_doc,
                        "max_q_doc_sim": max_q_doc,
                        "answer_context_sim": groundedness,
                        "answer_relevance_sim": answer_relevance_sim,
                        "hallucination_score": hallucination_score,
                        "coverage": coverage,
                        "answer_length": ans_len,
                        "context_precision_proxy": context_precision_proxy,
                        "context_recall_proxy": context_recall_proxy,
                    }

                    ss.history.append({
                        "role": "assistant",
                        "content": answer,
                        "metrics": metrics,
                        "is_fallback": (answer == FALLBACK_ANSWER),
                        "feedback": None,
                        "hallucination_risk": (groundedness < float(ss.grounding_threshold)),
                        "retrieved_docs": [{"content": d, "source": m.get("doc")} for d, m in zip(docs, meta)],
                        "retrieved_sims": sims,
                    })

                    st.markdown(answer)

# ---------- Dashboard Tab ----------
with tabs[1]:
    st.subheader("📈 Live Session Performance")
    assistant_turns = [m for m in ss.history if m["role"] == "assistant"]
    turn_count = len([m for m in ss.history if m["role"] == "user"])
    if turn_count == 0:
        st.info("Start chatting in the Chat tab to populate metrics.")
    else:
        fallback_count = sum(1 for m in assistant_turns if m.get("is_fallback"))
        feedbacks = [m.get("feedback") for m in assistant_turns if m.get("feedback") is not None]
        satisfaction_rate = (feedbacks.count("positive") / len(feedbacks) * 100) if feedbacks else 0.0
        latencies = [m.get("metrics", {}).get("total_latency_s", 0.0) for m in assistant_turns]
        avg_latency = np.mean(latencies) if latencies else 0.0

        col1, col2, col3, col4 = st.columns(4)
        col1.metric("Total Queries", f"{turn_count}")
        col2.metric("Avg. Latency (s)", f"{avg_latency:.2f}")
        col3.metric("Fallback Rate", f"{(fallback_count / turn_count * 100):.1f}%")
        col4.metric("User Satisfaction", f"{satisfaction_rate:.1f}%")

    st.markdown("---")

# ---------- Export Session Metrics ----------
def export_session_metrics_btn():
    if st.button("📥 Download Session Metrics CSV"):
        rows = []
        for msg in [m for m in ss.history if m["role"] == "assistant"]:
            r = {"answer": msg["content"], "is_fallback": msg.get("is_fallback", False)}
            r.update(msg.get("metrics", {}))
            rows.append(r)
        if not rows:
            st.info("No assistant turns yet.")
            return
        df = pd.DataFrame(rows)
        csv = df.to_csv(index=False).encode("utf-8")
        st.download_button("Download CSV", csv, "session_metrics.csv", "text/csv")

export_session_metrics_btn()



Overwriting app.py


In [ ]:
!ls -l

total 24
-rw-r--r-- 1 root root 16488 Sep 18 05:07 app.py
drwxr-xr-x 1 root root  4096 Sep 16 13:40 sample_data


## Streamlit Code for both user and developer

In [21]:
!pkill -f streamlit || echo "No old Streamlit process"
!pkill -f ngrok || echo "No old ngrok process"

^C
^C


In [22]:
import os, time, subprocess
from pyngrok import ngrok
from google.colab import userdata

# Secrets -> env
os.environ['MISTRAL_API_KEY'] = userdata.get('MISTRAL_API_KEY') or ""
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN') or ""

if not os.environ['MISTRAL_API_KEY']:
    print("❗ MISTRAL_API_KEY missing in Colab secrets.")
if not NGROK_AUTH_TOKEN:
    print("❗ NGROK_AUTH_TOKEN missing in Colab secrets.")

if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

APP_FILE = "app.py"

# Launch Streamlit
process = subprocess.Popen(["streamlit", "run", APP_FILE, "--server.port=8501"])
print("Starting Streamlit server...")
time.sleep(5)

public_url = ngrok.connect(8501)
print(f"✅ {APP_FILE} is live at: {public_url}")

Starting Streamlit server...
✅ app.py is live at: NgrokTunnel: "https://c64695acba98.ngrok-free.app" -> "http://localhost:8501"
